In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

### Ranking general

In [57]:
# Abrir parquet en un DataFrame
df_ranking = pd.read_parquet("../../data/raw/fbref_ranking_general.parquet")
df_final = df_ranking.copy()
# Codificacion equipo
# data = {'equipo': ["Real Madrid", "Barcelona", "Atlético", "Sevilla", "Betis",
#     "Villarreal", "Real Sociedad", "Valencia", "Osasuna", "Getafe",
#     "Rayo Vallecano", "Celta", "Alavés", "Las Palmas", "Mallorca",
#     "Granada", "Cádiz", "Girona", "Leganés", "Espanyol"]}

# encoder = OneHotEncoder(sparse_output=False)
# encoded = encoder.fit_transform(df_ranking[['Squad']])

# encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['Squad']))

# # Concatenar con el original (opcional)
# df_final = pd.concat([df_ranking, encoded_df], axis=1)
# df_final = df_final.drop(columns=['Squad']
df_ranking = df_ranking.drop(columns=['Notes', 'Attendance', 'Goalkeeper', 'Top Team Scorer'])

print(df_ranking.columns)


Index(['Rk', 'Squad', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts', 'Pts/MP',
       'xG', 'xGA', 'xGD', 'xGD/90', 'Last 5'],
      dtype='object')


### Ranking home/away

In [16]:
df_ranking_ha = pd.read_parquet("../../data/raw/fbref_ranking_home_away.parquet")

for col in df_ranking_ha.columns:
    col_clean = col.strip().replace("(", "").replace(")", "").replace("'", "")
    parts = [p.strip() for p in col_clean.split(",")]
    
    if len(parts) == 2:
        if parts[0] == "Home":
            new_col = parts[1] + "_home"
        elif parts[0] == "Away":
            new_col = parts[1] + "_away"
        else:
            new_col = parts[1]
    else:
        new_col = parts[0]
    
    df_ranking_ha = df_ranking_ha.rename(columns={col: new_col})

print(df_ranking_ha.head())

   Rk            Squad  MP_home  W_home  D_home  L_home  GF_home  GA_home  \
0   1        Barcelona        3       3       0       0       11        1   
1   2      Real Madrid        3       3       0       0        5        1   
2   3       Villarreal        4       4       0       0       10        1   
3   4            Elche        4       3       1       0        6        2   
4   5  Atlético Madrid        4       3       1       0       11        5   

   GD_home  Pts_home  ...  L_away  GF_away  GA_away  GD_away  Pts_away  \
0       10         9  ...       0       10        4        6        10   
1        4         9  ...       1       11        7        4         9   
2        9        12  ...       1        3        4       -1         4   
3        4        10  ...       0        4        4        0         3   
4        6        10  ...       1        3        4       -1         2   

   Pts/MP_away  xG_away  xGA_away  xGD_away  xGD/90_away  
0         2.50      7.8       4.9

### Players stats

In [46]:
# Abrir parquet en un DataFrame
df_players = pd.read_parquet("../../data/raw/fbref_players_stats.parquet")

# Crear player_id
df_players['player_id'] = df_players['Player'].str.replace(' ','')

# Eliminar columnas innecesarias
df_players = df_players.drop(columns=["Rk", "Nation", "Matches", "Player"])

print(df_players.head())

     Pos       Squad     Age  Born MP Starts  Min  90s Gls Ast  ... Ast-90min  \
0  MF,DF  Celta Vigo  19-256  2006  1      0   34  0.4   0   0  ...      0.00   
1     DF      Getafe  26-199  1999  3      3  252  2.8   0   0  ...      0.00   
2     FW     Sevilla  25-239  2000  4      3  286  3.2   0   1  ...      0.31   
3     DF       Elche  24-190  2001  5      5  450  5.0   0   0  ...      0.00   
4     GK    Valencia  24-273  2000  6      6  540  6.0   0   0  ...      0.00   

  G+A-90min G-PK-90min G+A-PK-90min xG-90min xAG-90min xG+xA-90min npxG-90min  \
0      0.00       0.00         0.00     0.00      0.14        0.14       0.00   
1      0.00       0.00         0.00     0.02      0.00        0.02       0.02   
2      0.31       0.00         0.31     0.31      0.03        0.34       0.31   
3      0.00       0.00         0.00     0.00      0.02        0.02       0.00   
4      0.00       0.00         0.00     0.00      0.00        0.00       0.00   

  npxG+xAG-90min          

### Results

In [47]:
# Abrir parquet en un DataFrame
df_matches = pd.read_parquet("../../data/raw/fbref_results.parquet")
# Crear la columna matches_id
df_matches['match_id'] = df_matches['Date'].astype(str).str.replace('-','') + '_' + df_matches['Home'].str[:2] + df_matches['Away'].str[:2]

# Eliminar columnas innecesarias
df_matches = df_matches.drop(columns=["Match Report", "Notes", "Venue", "Attendance"])
print(df_matches.head())

    Wk  Day        Date   Time        Home   xG Score  xG.1            Away  \
0  1.0  Fri  2025-08-15  19:00      Girona  0.6   1–3   3.4  Rayo Vallecano   
1  1.0  Fri  2025-08-15  21:30  Villarreal  2.4   2–0   0.9          Oviedo   
2  1.0  Sat  2025-08-16  19:30    Mallorca  0.2   0–3   2.1       Barcelona   
3  1.0  Sat  2025-08-16  21:30      Alavés  1.2   2–1   0.9         Levante   
4  1.0  Sat  2025-08-16  21:30    Valencia  2.0   1–1   0.8   Real Sociedad   

              Referee       match_id  
0     Javier Alberola  20250815_GiRa  
1     Alejandro Muñíz  20250815_ViOv  
2   José Luis Munuera  20250816_MaBa  
3        Miguel Sesma  20250816_AlLe  
4  Jose Maria Sánchez  20250816_VaRe  


### Squads stats

In [19]:
# Abrir parquet en un DataFrame
df_squad = pd.read_parquet("../../data/raw/fbref_squads_stats.parquet")

# Eliminar columnas innecesarias
# df_ranking = df_ranking.drop(columns=["Date", "Match Report", "Notes"])
print(df_squad.head())

  ('Unnamed: 0_level_0', 'Squad')  ('Unnamed: 1_level_0', '# Pl')  \
0                          Alavés                              22   
1                   Athletic Club                              23   
2                 Atlético Madrid                              21   
3                       Barcelona                              22   
4                           Betis                              25   

   ('Unnamed: 2_level_0', 'Age')  ('Unnamed: 3_level_0', 'Poss')  \
0                           27.8                            49.5   
1                           27.7                            52.7   
2                           27.8                            59.3   
3                           24.9                            69.6   
4                           27.9                            50.2   

   ('Playing Time', 'MP')  ('Playing Time', 'Starts')  \
0                       6                          66   
1                       6                          66   
2    

### Player-matches

In [44]:
# Abrir parquet en un DataFrame
df_matches_players = pd.read_parquet("../../data/raw/fbref_player_matches.parquet")

# Crear match_id
df_matches_players['match_id'] = df_matches_players.apply(
    lambda x: x['Date'].replace('-','') + '_' + 
    (x['Squad'][:2] + x['Opponent'][:2] if x['Venue'] == 'Home' else x['Opponent'][:2] + x['Squad'][:2]), 
    axis=1
)

# Crear player_id
df_matches_players['player_id'] = df_matches_players['Player'].str.replace(' ','')

# Eliminar columnas innecesarias
df_matches_players = df_matches_players.drop(columns=["Date", "Match Report", "Comp", "PlayerLink", 'Player'])

print(df_matches_players.head())

   Day        Round Venue Result       Squad    Opponent Start  \
0  Sun  Matchweek 1  Home  L 0–2  Celta Vigo      Getafe     N   
1  Sat  Matchweek 2  Away  D 1–1  Celta Vigo    Mallorca     N   
2  Wed  Matchweek 6  Home  D 1–1  Celta Vigo       Betis     N   
3  Sun  Matchweek 3  Home  D 1–1  Celta Vigo  Villarreal     N   
4  Sun  Matchweek 4  Home  D 1–1  Celta Vigo      Girona     N   

                                   Pos                                  Min  \
0                                RM,RB                                   34   
1  On matchday squad, but did not play  On matchday squad, but did not play   
2  On matchday squad, but did not play  On matchday squad, but did not play   
3  On matchday squad, but did not play  On matchday squad, but did not play   
4  On matchday squad, but did not play  On matchday squad, but did not play   

                       Performance_Gls  ...  \
0                                    0  ...   
1  On matchday squad, but did not 

### Mergeado de tablas

In [61]:
df = df_matches_players.merge(df_matches, on="match_id")
df = df.merge(df_ranking.add_prefix("team_"), left_on="Squad", right_on="team_Squad", how="left")
df = df.merge(df_ranking.add_prefix("opp_"), left_on="Opponent", right_on="opp_Squad", how="left")
df = df.merge(df_players, on="player_id")

print(df.columns)
print(df.head())

Index(['Day_x', 'Round', 'Venue', 'Result', 'Squad_x', 'Opponent', 'Start',
       'Pos_x', 'Min_x', 'Performance_Gls',
       ...
       'Gls-90min', 'Ast-90min', 'G+A-90min', 'G-PK-90min', 'G+A-PK-90min',
       'xG-90min', 'xAG-90min', 'xG+xA-90min', 'npxG-90min', 'npxG+xAG-90min'],
      dtype='object', length=111)
  Day_x        Round Venue Result     Squad_x    Opponent Start  \
0   Sun  Matchweek 1  Home  L 0–2  Celta Vigo      Getafe     N   
1   Sat  Matchweek 2  Away  D 1–1  Celta Vigo    Mallorca     N   
2   Wed  Matchweek 6  Home  D 1–1  Celta Vigo       Betis     N   
3   Sun  Matchweek 3  Home  D 1–1  Celta Vigo  Villarreal     N   
4   Sun  Matchweek 4  Home  D 1–1  Celta Vigo      Girona     N   

                                 Pos_x                                Min_x  \
0                                RM,RB                                   34   
1  On matchday squad, but did not play  On matchday squad, but did not play   
2  On matchday squad, but did not play 

In [64]:
# Guardar en excel, para visualizar
df.to_excel("../../data/processed/combined_data.xlsx", index=False)